In [121]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lotteria:

    def __init__(self,
                 max_numbers=90,
                 max_extra=90,
                 len_numbers=6,
                 len_extra=1):
        self.max_numbers = max_numbers
        self.max_extra = max_extra
        self.len_numbers = len_numbers
        self.len_extra = len_extra
        self.combo = None
        self.extra = None

    @property
    def backend(self):
        return self._backend

    @property
    def many(self):
        return self._many

    @staticmethod
    def pallottoliere(max):
        while True:
            yield rnd.randint(1, max)

    @staticmethod
    def choice(len, max):
        whole_numbers = list(range(1, max+1))

        def extraction():
            sample = rnd.choice(whole_numbers)
            whole_numbers.remove(sample)

            return sample

        return frozenset((extraction() for _ in range(len)))

    @staticmethod
    def sample(len, max):
        whole_numbers = tuple(range(1, max))

        return frozenset(
            rnd.sample(whole_numbers, k=len))

    def randint(self, _len, max):
        combo = set()
        while len(combo) < _len:
            combo.add(next(self.pallottoliere(max)))

        return frozenset(combo)

    def manySamples(self, extract):
        size = self._many or 1

        def sampler(backend):
            samples = (extract(backend) for _ in range(size))

            return rnd.choice(tuple(samples))

        return sampler

    def extract(self, backend):
        combo = backend(self.len_numbers, self.max_numbers)
        extra = backend(self.len_extra, self.max_extra) or None

        return combo, extra

    def __call__(self, backend='sample', many=1):
        self._backend = backend
        self._many = many

        backend = eval(backend,
                       {'__builtins__': {}},
                       {'choice': self.choice,
                        'randint': self.randint,
                        'sample': self.sample})

        self.combo, self.extra = self.manySamples(self.extract)(backend)

        #return self._combo, self._extra

    def draw(self, backend='sample', many=1):
        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        self(backend, many)

        print('Estrazione del:', now, '\nNumeri Estratti:',
              *sorted(self.combo))  # type: ignore
        if self.extra is not None:
            print('Superstar:', *sorted(self.extra))  # type: ignore

In [118]:
superenalotto = Lotteria(max_numbers=90, max_extra=90, len_numbers=6, len_extra=1)
superenalotto.draw(backend='choice', many=100_000)

Estrazione del: 18/06/2023 12:33 
Numeri Estratti: 9 15 17 43 67 68
Superstar: 32


In [119]:
eurojack = Lotteria(max_numbers=50, max_extra=12, len_numbers=5, len_extra=2)
eurojack.draw(backend='sample', many=100_000)

Estrazione del: 18/06/2023 12:33 
Numeri Estratti: 15 18 26 33 49
Superstar: 6 9


In [104]:
winForLife = Lotteria(20, 20, 10, 1)
winForLife.draw(backend='sample', many=100_000)

Estrazione del: 18/06/2023 12:20 
Numeri Estratti: 3 4 5 7 9 10 13 16 18 19
Superstar: 9


In [106]:
superenalotto.len_extra = 1
superenalotto.draw()

Estrazione del: 18/06/2023 12:20 
Numeri Estratti: 9 21 29 32 36 83
Superstar: 17


In [113]:
superenalotto.draw(backend='randint')

Estrazione del: 18/06/2023 12:23 
Numeri Estratti: 11 24 45 77 85 89
Superstar: 39


In [70]:
superenalotto.len_extra

1

In [71]:
superenalotto.many

1

In [72]:
# trying to set backend via attribute: fails as expected
winForLife.backend = 'int'

AttributeError: property 'backend' of 'Lotteria' object has no setter

In [ ]:
# for b in ['choice', 'int', 'sample']:
#    print('######')
#    eurojack(backend=b, many=0)
#    print('backend:', eurojack.backend)
#
# print('\n################# many ####################\n')
#
# for b in ['choice', 'int', 'sample']:
#    print('######')
#    eurojack(backend=b)
#    print('backend:', eurojack.backend)

In [123]:
win = Lotteria(20, 20, 10, 1)

w_1 = frozenset({2, 3, 5, 7, 9, 11, 12, 14, 16, 18})
w_2 = frozenset({7})

In [125]:
i = 1
win()
x_1, x_2 = win.combo, win.extra
while ((w_1 != x_1) and (not w_1.isdisjoint(x_1))) or (w_2 != x_2):
    win()
    x_1, x_2 = win.combo, win.extra
    i += 1
    if i % 1_000_000 == 0:
        print(i)
print(i, '>>>', x_1, x_2)

1000000
1119905 >>> frozenset({2, 3, 5, 7, 9, 11, 12, 14, 16, 18}) frozenset({7})


In [126]:
win.__dict__

{'max_numbers': 20,
 'max_extra': 20,
 'len_numbers': 10,
 'len_extra': 1,
 'combo': frozenset({2, 3, 5, 7, 9, 11, 12, 14, 16, 18}),
 'extra': frozenset({7}),
 '_backend': 'sample',
 '_many': 1}